# <font color=#c51b8a>VPOD 'Mine-n-Match':</font>
## <font color=#c994c7>Part 1 Objective</font> - Use Species Names from Microspectrophotemetry Data Sheet to Query NCBI for All Related Opsin Sequences  

In [1]:
import pandas as pd
from deepBreaks.preprocessing import read_data
from Bio import Entrez, SeqIO
import re #regular expressions
import time
import mysql
import mysql.connector

In [4]:
scp_file = 'c:/Users/safra/Documents/GitHub/visual-physiology-opsin-db/scripts_n_notebooks/vpod_ML_workflows/subtests/work_in_progress/mine_n_match/scp_raw.tsv'
scp_df = read_data(scp_file, seq_type = None, is_main=False)

In [5]:
scp_df.head()

,Genus,Species,phylum,CellType,CellSubType,LambdaMax,error,Chromophore,Stage,useable,expid,opsinid,refid,Notes,Unnamed: 15,isGenusInHetero,isSpeciesInHetero,isInHetero,isVert
maxid,,,,,,,,,,,,,,,,,,,
1,Acipenser,transmontanus,Chordata,rod,NaN,540.0,NaN,NaN,adult,1.0,NaN,463,30,partial cds but is rod opsin,NaN,NaN,NaN,0,1.0
2,Acipenser,transmontanus,Chordata,cone,NaN,464.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN
3,Acipenser,transmontanus,Chordata,cone,NaN,531.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN
4,Acipenser,transmontanus,Chordata,cone,NaN,605.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN
5,Acipenser,medirostris,Chordata,rod,NaN,540.0,NaN,NaN,adult,0.0,NaN,0,43,NaN,NaN,NaN,NaN,0,NaN


In [6]:
species_list = []
for sp, genus in zip(scp_df['Genus'], scp_df['Species']):
    full_species = f'{sp} {genus}'
    species_list.append(full_species)
len(species_list)

1570

In [17]:
unique_species_list = list(set(species_list)) 
len(unique_species_list)

469

In [8]:
# create a function that return the collection of all NCBI fetch result

def ncbi_fetch(email, term, ncbi_db = "nuccore", rettype = "gb", format = "genbank"):
  Entrez.email = email    # Always tell NCBI who you are
  handle = Entrez.esearch(db=ncbi_db,
                        term=term, 
                        api_key = "1efb120056e1cea873ba8d85d6692abd5d09") # using api key allows 10 fetch per second
  record = Entrez.read(handle)
  #print(record)
  full_res = int(record["Count"])
  
  handle_full = Entrez.esearch(db=ncbi_db,
                        term=term, 
                        api_key = "1efb120056e1cea873ba8d85d6692abd5d09",
                        retmax = full_res + 1)
  record_full = Entrez.read(handle_full)
  print(record_full)
  q_list = record_full["IdList"]

# create a list for all the sequence fetched from NCBI
  NCBI_seq =[]
  for x in q_list:
    # fetch result from previous search
    fet = Entrez.efetch(db=ncbi_db, id=x, rettype=rettype)
    seq = SeqIO.read(fet, format)
    fet.close()
    
    NCBI_seq.append(seq)

    time.sleep(0.5)
  
  return NCBI_seq
  

In [9]:
scp_query_list = []
for species in unique_species_list:
    NCBI_seq = ncbi_fetch("sethfrazer@ucsb.edu", 
                    f"{species} AND (opsin[All Fields] AND complete[All Fields] AND cds[All Fields] NOT voucher)",)
    scp_query_list.append(NCBI_seq)
    

{'Count': '12', 'RetMax': '12', 'RetStart': '0', 'IdList': ['284822155', '57164785', '57164775', '57164773', '57164761', '57164759', '56548587', '56548547', '56548527', '56548507', '56548487', '282158242'], 'TranslationSet': [{'From': 'Pundamilia nyererei', 'To': '"Pundamilia nyererei"[Organism] OR Pundamilia nyererei[All Fields]'}], 'TranslationStack': [{'Term': '"Pundamilia nyererei"[Organism]', 'Field': 'Organism', 'Count': '111519', 'Explode': 'Y'}, {'Term': 'Pundamilia nyererei[All Fields]', 'Field': 'All Fields', 'Count': '111649', 'Explode': 'N'}, 'OR', 'GROUP', {'Term': 'opsin[All Fields]', 'Field': 'All Fields', 'Count': '45609', 'Explode': 'N'}, {'Term': 'complete[All Fields]', 'Field': 'All Fields', 'Count': '63837211', 'Explode': 'N'}, 'AND', {'Term': 'cds[All Fields]', 'Field': 'All Fields', 'Count': '188635857', 'Explode': 'N'}, 'AND', {'Term': 'voucher[All Fields]', 'Field': 'All Fields', 'Count': '12765102', 'Explode': 'N'}, 'NOT', 'GROUP', 'AND'], 'QueryTranslation': '

In [27]:
# create empty lists
Accession = []
DNA = []
Genus = []
Species = []
gene_des = []
version = []
Protein = []
full_sp_names = []
# loop through the result list obtained from the NCBI search
# may take over 10 minutes
for query in scp_query_list:
    for seq in query:
        # get genus nd speceis name seperately
        spe_name = seq.annotations["organism"]
        g_s_name = spe_name.split()

        # get and append protein sequence
        if seq.features:
            for feature in seq.features:
                if feature.type == "CDS":
                    if "translation" in feature.qualifiers.keys():
                        pro_seq = feature.qualifiers['translation'][0]

        # attached them to lists
        Accession.append(str(seq.name))
        Genus.append(str(g_s_name[0]))
        Species.append(str(g_s_name[1]))
        full_sp_names.append(str(g_s_name[0]) + '_' + str(g_s_name[1]))
        gene_des.append(str(seq.description))
        version.append(str(seq.id))
        Protein.append(str(pro_seq))


In [28]:
# create a dataframe for the information
ncbi_q_op = pd.DataFrame(
    {'Accession': version,
     'Genus': Genus,
     'Species': Species,
     'full_sp_name': full_sp_names,
     'Protein': Protein,
     'gene_des': gene_des
    })

In [29]:
ncbi_q_op.head()

,Genus,Species,Accession,Protein,gene_des,version
0,Pundamilia,nyererei,GQ422482,MAWDGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYMVDPIIYK...,Pundamilia nyererei green sensitive visual pig...,GQ422482.1
1,Pundamilia,nyererei,AY673768,MNGTEGPFFYVPMVNTTGIVRSPYEYPQHYLVSPAAYAALGCYMFF...,Pundamilia nyererei isolate MSP #3 rhodopsin (...,AY673768.1
2,Pundamilia,nyererei,AY673763,MKGKRDMELPEDFWIPVSLDTNNITSLSPFLVPQDHLGDTATFYAM...,Pundamilia nyererei isolate MSP #8 putative bl...,AY673763.1
3,Pundamilia,nyererei,AY673762,MKGKRDMELPEDFWIPVSLDTNNITSLSPFLVPQDHLGDTATFYAM...,Pundamilia nyererei isolate MSP #3 putative bl...,AY673762.1
4,Pundamilia,nyererei,AY673756,MAWEGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYLADPIFFK...,Pundamilia nyererei isolate MSP #8 putative gr...,AY673756.1


In [30]:
ncbi_q_op.to_csv(path_or_buf="ncbi_q_op.csv", index=False)

## <font color=#c994c7>Part 1 Objective</font> - Query OPTICS with all sequence data to obtain lmax predictions then match the sequence to it's closest MSP value based on those predictions

We'll need to...

- Format all sequences to a FASTA file from the existing df
- Query OPTICS with all the sequence data [bootstrap enabled]
- Extract Predictions
- Match to closest MSP value species-by-species [will need a list of the unique species names] // Match to MaxId as the foreign key

In [1]:
import pandas as pd
from deepBreaks.preprocessing import read_data

In [2]:
# Loading mined seq data and cleaning it of all redundant entries (cases where Species and Sequence are redundant - we keep cases where identical sequence is present in different species) 
seqFileData = 'c:/Users/safra/Documents/GitHub/visual-physiology-opsin-db/scripts_n_notebooks/vpod_ML_workflows/subtests/work_in_progress/mine_n_match/ncbi_q_op.csv'
seq_data = read_data(seqFileData, seq_type = None, is_main=False)
seq_data['full_species'] = seq_data['Genus'] + '_' + seq_data['Species']
seq_data_filtered = seq_data.copy().drop_duplicates(subset=['full_species', 'Protein'])
seq_data_filtered.to_csv(path_or_buf="ncbi__filtered_q_op.csv", index=True)
seq_data_filtered.head()

,Genus,Species,Protein,gene_des,full_species
Accession,,,,,
GQ422482.1,Pundamilia,nyererei,MAWDGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYMVDPIIYK...,Pundamilia nyererei green sensitive visual pig...,Pundamilia_nyererei
AY673768.1,Pundamilia,nyererei,MNGTEGPFFYVPMVNTTGIVRSPYEYPQHYLVSPAAYAALGCYMFF...,Pundamilia nyererei isolate MSP #3 rhodopsin (...,Pundamilia_nyererei
AY673763.1,Pundamilia,nyererei,MKGKRDMELPEDFWIPVSLDTNNITSLSPFLVPQDHLGDTATFYAM...,Pundamilia nyererei isolate MSP #8 putative bl...,Pundamilia_nyererei
AY673756.1,Pundamilia,nyererei,MAWEGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYLADPIFFK...,Pundamilia nyererei isolate MSP #8 putative gr...,Pundamilia_nyererei
AY673738.1,Pundamilia,nyererei,MNGTEGPFFYVPMVNTTGIVRSPYEYPQHYLVSPAAYAALGCYMFF...,Pundamilia nyererei rod visual pigment (Rod) g...,Pundamilia_nyererei


In [39]:
fasta_file = 'msp_mined_seqs.fasta'
with open(fasta_file, 'w') as f:
    for id, seq in zip(seq_data_filtered.index.to_list(), seq_data_filtered['Protein']):
        f.write(f'>{id}\n{seq}\n')

In [5]:
# Example use
# python optics_predictions.py -in msp_mined_seqs.fasta -rd mined_msp_seqs -out mined_seq_predictions.tsv -m wildtype -e aa_prop -b True -ir msp_mined_seq_blastp_report.tsv -r bovine -s False -bsv msp_bs_viz.pdf
predFileData = './optics_on_mined_msp_seqs_2024-09-25_18-21-34/mined_seq_predictions.tsv'
pred_df = read_data(predFileData, seq_type = None, is_main=False)
pred_df.head()

,Single_Prediction,Prediction_Means,Prediction_Medians,Prediction_Lower_Bounds,Prediction_Upper_Bounds,Std_Deviation,%Identity_Nearest_VPOD_Sequence
Names,,,,,,,
GQ422482.1,526.2,524.3,525.0,517.1,527.8,3.2,97.159
AY673768.1,496.2,496.8,497.0,492.5,500.2,2.1,95.763
AY673763.1,453.6,453.1,454.5,438.1,464.8,7.0,81.481
AY673756.1,523.8,520.7,521.2,514.7,525.3,3.0,96.307
AY673738.1,496.1,496.8,496.8,492.6,500.1,2.1,95.48


In [13]:
mnm_merged_df = pd.merge(seq_data_filtered, pred_df, left_index=True, right_index=True)
mnm_merged_df.head()

,Genus,Species,Protein,gene_des,full_species,Single_Prediction,Prediction_Means,Prediction_Medians,Prediction_Lower_Bounds,Prediction_Upper_Bounds,Std_Deviation,%Identity_Nearest_VPOD_Sequence
Accession,,,,,,,,,,,,
GQ422482.1,Pundamilia,nyererei,MAWDGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYMVDPIIYK...,Pundamilia nyererei green sensitive visual pig...,Pundamilia_nyererei,526.2,524.3,525.0,517.1,527.8,3.2,97.159
AY673768.1,Pundamilia,nyererei,MNGTEGPFFYVPMVNTTGIVRSPYEYPQHYLVSPAAYAALGCYMFF...,Pundamilia nyererei isolate MSP #3 rhodopsin (...,Pundamilia_nyererei,496.2,496.8,497.0,492.5,500.2,2.1,95.763
AY673763.1,Pundamilia,nyererei,MKGKRDMELPEDFWIPVSLDTNNITSLSPFLVPQDHLGDTATFYAM...,Pundamilia nyererei isolate MSP #8 putative bl...,Pundamilia_nyererei,453.6,453.1,454.5,438.1,464.8,7.0,81.481
AY673756.1,Pundamilia,nyererei,MAWEGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYLADPIFFK...,Pundamilia nyererei isolate MSP #8 putative gr...,Pundamilia_nyererei,523.8,520.7,521.2,514.7,525.3,3.0,96.307
AY673738.1,Pundamilia,nyererei,MNGTEGPFFYVPMVNTTGIVRSPYEYPQHYLVSPAAYAALGCYMFF...,Pundamilia nyererei rod visual pigment (Rod) g...,Pundamilia_nyererei,496.1,496.8,496.8,492.6,500.1,2.1,95.48


In [51]:
scp_file = 'c:/Users/safra/Documents/GitHub/visual-physiology-opsin-db/scripts_n_notebooks/vpod_ML_workflows/subtests/work_in_progress/mine_n_match/scp_raw.tsv'
scp_df = read_data(scp_file, seq_type = None, is_main=False)
sp_list = []
og_sp_list = scp_df['Species'].to_list()
for sp in og_sp_list:
    sp_list.append(sp.split(' ')[0])
scp_df['full_species'] = scp_df['Genus'] + '_' + sp_list
scp_df.head()

,Genus,Species,phylum,CellType,CellSubType,LambdaMax,error,Chromophore,Stage,useable,expid,opsinid,refid,Notes,Unnamed: 15,isGenusInHetero,isSpeciesInHetero,isInHetero,isVert,full_species
maxid,,,,,,,,,,,,,,,,,,,,
1,Acipenser,transmontanus,Chordata,rod,NaN,540.0,NaN,NaN,adult,1.0,NaN,463,30,partial cds but is rod opsin,NaN,NaN,NaN,0,1.0,Acipenser_transmontanus
2,Acipenser,transmontanus,Chordata,cone,NaN,464.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN,Acipenser_transmontanus
3,Acipenser,transmontanus,Chordata,cone,NaN,531.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN,Acipenser_transmontanus
4,Acipenser,transmontanus,Chordata,cone,NaN,605.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN,Acipenser_transmontanus
5,Acipenser,medirostris,Chordata,rod,NaN,540.0,NaN,NaN,adult,0.0,NaN,0,43,NaN,NaN,NaN,NaN,0,NaN,Acipenser_medirostris


In [ ]:
# use the hungarian algorthym?
# maybe use the std deviation as a basis for confidence in the match as well?

In [20]:
# Get unique species from predictions
unique_species = list(mnm_merged_df['full_species'].unique())
len(unique_species)

121

In [ ]:
# Assuming your dataframes are structured as follows:
# predictions_df: columns include 'species', 'accession', 'prediction_value'
# measurements_df: columns include 'species', 'measurement_value'

# Initialize a list to store the matched results
matched_results = []

# Iterate through each species
for species in unique_species:

  # Filter predictions and measurements for the current species
  species_predictions = mnm_merged_df[mnm_merged_df['full_species'] == species]
  species_measurements = scp_df[scp_df['full_species'] == species]

  # Iterate through each prediction for the current species
  for _, prediction_row in species_predictions.iterrows():
    prediction_value = prediction_row['Prediction_Means']
    try:
      accession = prediction_row['Accession']
    except:
      accession = prediction_row.name

    # Calculate absolute differences between the prediction and all measurements for the species
    species_measurements['abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()

    # Find the closest measurement (handling ties)
    closest_measurement_row = species_measurements.sort_values('abs_diff').iloc[0]
    min_diff = closest_measurement_row['abs_diff']
    # Store the matched result
    matched_results.append({
      'accession': accession,
      'species': species,
      'prediction_value': prediction_value,
      'closest_measurement': closest_measurement_row['LambdaMax'],
      'abs_diff' : min_diff,
      'max_id': closest_measurement_row.name 
    })

# Create a new dataframe from the matched results
final_matched_df = pd.DataFrame(matched_results)

print(final_matched_df)

In [57]:
# Assuming your dataframes are structured as follows:
# predictions_df: columns include 'species', 'accession', 'prediction_value'
# measurements_df: columns include 'species', 'measurement_value'

# Initialize a list to store the matched results
matched_results = []
i = 0
# Iterate through each species
for species in unique_species:
  scp_df_copy = scp_df.copy()
  # Filter predictions and measurements for the current species
  species_predictions = mnm_merged_df[mnm_merged_df['full_species'] == species]
  species_measurements = scp_df_copy[scp_df_copy['full_species'] == species]
  if species_measurements.shape[0] == 0:
    i+=1
    print(f'\nThis species is missing a match: {species}\n')
  else:
      #print(species)
    # Iterate through each prediction for the current species
    for _, prediction_row in species_predictions.iterrows():
      prediction_value = prediction_row['Prediction_Means']
      try:
        accession = prediction_row['Accession']
      except:
        accession = prediction_row.name

      # Calculate absolute differences between the prediction and all measurements for the species
      species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
      # Find the closest measurement (handling ties)
      closest_measurement_row = species_measurements.sort_values('abs_diff').iloc[0]
      min_diff = closest_measurement_row['abs_diff']
      
      existing_match_index = next((i for i, match in enumerate(matched_results) if match['Accession'] == accession), None)

      if (existing_match_index is not None): 
        if (min_diff < matched_results[existing_match_index]['abs_diff']):
          # If it exists, compare the absolute differences and keep the better match
          matched_results[existing_match_index].update({
              'prediction_value': prediction_value,
              'closest_measurement': closest_measurement_row['LambdaMax'],
              'abs_diff': min_diff,
              'max_id': closest_measurement_row.name
          })
      else:
          # If it doesn't exist, add the new match
          matched_results.append({
              'Accession': accession,
              'full_species': species,
              'prediction_value': prediction_value,
              'closest_measurement': closest_measurement_row['LambdaMax'],
              'abs_diff': min_diff,
              'max_id': closest_measurement_row.name
          })
        
# Create a new dataframe from the matched results
final_matched_df = pd.DataFrame(matched_results)

print(f'There were {i} unmatched species')
print(final_matched_df.head())

C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWit


This species is missing a match: Maylandia_pyrsonotos


This species is missing a match: Maylandia_phaeos


This species is missing a match: Maylandia_mbenjii


This species is missing a match: Maylandia_lombardoi


This species is missing a match: Pseudotropheus_fainzilberi


This species is missing a match: Maylandia_callainos


This species is missing a match: Maylandia_benetos


This species is missing a match: Maylandia_barlowi


This species is missing a match: synthetic_construct


This species is missing a match: Aspergillus_niger


This species is missing a match: Cotesia_congregata


This species is missing a match: Penicillium_rubens


This species is missing a match: Gallus_gallus


This species is missing a match: Ostorhinchus_doederleini


This species is missing a match: Columba_livia


This species is missing a match: Cololabis_saira



C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWit


This species is missing a match: Cephalopachus_bancanus


This species is missing a match: Carlito_syrichta


This species is missing a match: Bodianus_mesothorax


This species is missing a match: Zoramia_leptacanthus


This species is missing a match: Nectamia_savayensis



C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWit


This species is missing a match: Micropoecilia_bifurca


This species is missing a match: Sardinops_melanosticta



C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWit


This species is missing a match: Acrossocheilus_paradoxus


This species is missing a match: Isoodon_obesulus


This species is missing a match: Ostorhinchus_cyanosoma


This species is missing a match: Pundamilia_sp.


This species is missing a match: Pseudotropheus_sp.



C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
C:\Users\safra\AppData\Local\Temp\ipykernel_40172\3046422808.py:28: SettingWit


This species is missing a match: Psalidodon_fasciatus


This species is missing a match: Astyanax_mexicanus


This species is missing a match: Tharsalea_rubidus


This species is missing a match: Bombus_impatiens


This species is missing a match: Ostorhinchus_cookii


This species is missing a match: Rhabdamia_gracilis


This species is missing a match: Zoramia_viridiventer


This species is missing a match: Taeniopygia_guttata


This species is missing a match: Coturnix_japonica


This species is missing a match: Aristostomias_scintillans


This species is missing a match: Aristostomias_tittmanni


This species is missing a match: Gekko_chinensis


This species is missing a match: Gekko_japonicus


This species is missing a match: Gekko_swinhonis


This species is missing a match: Gekko_scabridus


This species is missing a match: Gekko_hokouensis


This species is missing a match: Gekko_auriverrucosus


This species is missing a match: Gekko_gecko

There were 46 unmatched species
 

In [59]:
final_matched_df.shape

(474, 6)

In [61]:
len(final_matched_df['Accession'].unique())

474

In [80]:
len(final_matched_df['max_id'].unique())

187

In [82]:
iden_list = []
for _, d in final_matched_df.iterrows():
    #print(d)
    acc = d['Accession']
    iden_list.append(pred_df.loc[acc]['%Identity_Nearest_VPOD_Sequence'])
    
final_matched_df['%Identity_Nearest_VPOD_Sequence'] = iden_list

Once we have the dataframe of matched seqs - each accession is matched to an id - but some IDs may have several matches... 
At this point we do another 2 step filter

1. Filter by abs. error - keeping the accession with the lowest abs. error
2. If there is a tie - then filter by percent identitiy to sequences in VPOD 

In [96]:
import pandas as pd

# Group by 'max_id' and count unique accessions
grouped_counts = final_matched_df.groupby('max_id')['Accession'].nunique()

# Filter groups with more than one unique accession
duplicate_max_id_groups = grouped_counts[grouped_counts > 1]

if not duplicate_max_id_groups.empty:
    filtered_results = []

    # Iterate through each max_id with duplicates
    for max_id in duplicate_max_id_groups.index:
        # Filter rows with the current max_id
        duplicates = final_matched_df[final_matched_df['max_id'] == max_id]

        # Sort by abs_diff (ascending) and then percent_identity (descending)
        duplicates = duplicates.sort_values(['abs_diff', 'Accession'], ascending=[True, False])
        duplicates = duplicates.sort_values(['abs_diff', '%Identity_Nearest_VPOD_Sequence'], ascending=[True, False])

        # Keep only the first row (lowest abs_diff, highest percent_identity if tied)
        filtered_results.append(duplicates.iloc[0])

    # Combine filtered results with non-duplicate rows
    non_duplicates = final_matched_df[~final_matched_df['max_id'].isin(duplicate_max_id_groups.index)]
    final_filtered_df = pd.concat([pd.DataFrame(filtered_results), non_duplicates], ignore_index=True)
    final_filtered_df = final_filtered_df.sort_values(['abs_diff', '%Identity_Nearest_VPOD_Sequence'], ascending=[True, False])
    final_filtered_df.reset_index(drop=True, inplace=True)
else:
    final_filtered_df = final_matched_df 

final_filtered_df.head()

,Accession,full_species,prediction_value,closest_measurement,abs_diff,max_id,%Identity_Nearest_VPOD_Sequence
0,DQ168661.1,Poecilia_reticulata,560.0,560.0,0.0,1274,98.034
1,DQ088628.1,Melanochromis_vermivorus,555.0,555.0,0.0,389,93.838
2,L11865.1,Carassius_auratus,509.0,509.0,0.0,1349,90.544
3,AY745192.1,Setonix_brachyurus,538.0,538.0,0.0,1475,86.686
4,MW219662.1,Myripristis_berndti,495.1,495.0,0.1,67,98.281


In [97]:
final_filtered_df.shape

(187, 7)

In [99]:
len(final_filtered_df['full_species'].unique())

75

In [98]:
final_filtered_df.to_csv(path_or_buf="mnm_results_v1.csv", index=True)